# Print pokemons

api: https://github.com/rubenvereecken/pokemongo-api

In [ ]:
import print_pokemons as pp
from pogo.pokedex import pokedex

In [ ]:
username, password = pp.get_credentials_from_keyfile('keyfile')

In [ ]:
trainer = pp.get_trainer(username, password)

In [ ]:
bag = pp.get_pokemons(trainer)

In [ ]:
pp.print_pokemons_by_iv(bag)

## Count pokemons and possible evolves

In [ ]:
from collections import Counter

counter = Counter([(x['name'], x['id']) for x in bag])

print(counter.most_common(5))

In [ ]:
inventory = trainer.session.getInventory()

In [ ]:
print('{:>15} | {:>5} | {:>10} | {}'.format('Name', 'Num', 'Candies', 'Possible Evolutions'))
print('-'*60)
for item, count in counter.most_common(5):
    n_candies = inventory.candies.get(pokedex.families[item[1]], 0)
    possible = int(n_candies / count)
    print('{:>15} | {:>5} | {:>10} | {}'.format(item[0], count, n_candies, possible))